In [1]:
import os
import json
import seaborn as sns
import matplotlib .pyplot as plt
from sklearn .model_selection import train_test_split
import numpy as np

In [2]:
file_path = 'D:\\Dataset\\#4_전력 설비 에너지 패턴 및 고장 분석 센서\\Training\\1.펌프_일반모터\\2.SOH진단\\1.역률평균\\0.Combined\\'
files = ['Combined_LabelledData_346_역률평균.json', 'Combined_LabelledData_348_역률평균.json']
x_label = []
y_label = []

for file_name in files:
    full_path = file_path + file_name
    with open(full_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for entry in data['data']:
        if entry['ITEM_NAME'] == "역률평균":
            x_label.append(entry['ITEM_VALUE'])
            if entry['LABEL_NAME']=='정상':
                y_label.append(0)
            elif entry['LABEL_NAME']=='주의':
                y_label.append(1) 
            else:
                y_label.append(2)

x_label = np.array(x_label)
x_label_scaled = (x_label - np.mean(x_label)) / np.std(x_label)



In [3]:
data_indices = np.arange(len(x_label))
print(data_indices)
np.random.shuffle(data_indices)
print(data_indices)

x_label = np.array(x_label)[data_indices]
y_label = np.array(y_label)[data_indices]

x_train, x_temp, y_train, y_temp = train_test_split(x_label, y_label, test_size=0.4, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)

print(f"Train size: {len(x_train)}")
print(f"Validation size: {len(x_valid)}")
print(f"Test size: {len(x_test)}")

[    0     1     2 ... 57530 57531 57532]
[33572 15937 20466 ... 14083 33853 52790]
Train size: 34519
Validation size: 11507
Test size: 11507


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model, load_model

model_dnn = Sequential()
model_dnn.add(Dense(64, activation='relu', input_shape=(1,))) 
model_dnn.add(Dense(64, activation='relu'))
model_dnn.add(Dense(1)) 

model_dnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

x_train = x_train.reshape((x_train.shape[0], 1))

model_dnn.fit(x_train, y_train, epochs=100, batch_size=32)

model_dnn.save('dnn_model.h5')

Epoch 1/100
1079/1079 [==============================] - 1s 939us/step - loss: 0.1346 - accuracy: 0.5343
Epoch 2/100
1079/1079 [==============================] - 1s 915us/step - loss: 0.0089 - accuracy: 0.5703
Epoch 3/100
1079/1079 [==============================] - 1s 921us/step - loss: 0.0081 - accuracy: 0.5704
Epoch 4/100
1079/1079 [==============================] - 1s 936us/step - loss: 0.0073 - accuracy: 0.5705
Epoch 5/100
1079/1079 [==============================] - 1s 927us/step - loss: 0.0068 - accuracy: 0.5706
Epoch 6/100
1079/1079 [==============================] - 1s 942us/step - loss: 0.0066 - accuracy: 0.5706
Epoch 7/100
1079/1079 [==============================] - 1s 955us/step - loss: 0.0066 - accuracy: 0.5705
Epoch 8/100
 857/1079 [======================>.......] - ETA: 0s - loss: 0.0062 - accuracy: 0.5720

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix

# 테스트 데이터로 예측
x_test_reshaped = x_test.reshape((x_test.shape[0], 1))
y_pred = model_dnn.predict(x_test_reshaped)

# 이진 분류일 경우
y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]

# Confusion matrix 출력
cm = confusion_matrix(y_test, y_pred_binary)
print('Confusion Matrix:')
print(cm)


360/360 [==============================] - 0s 727us/step
Confusion Matrix:
[[2294    0    0]
 [   0 4993    0]
 [   0 4220    0]]
